In [1]:
import matplotlib.pyplot as plt
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, LabelBinarizer
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd
from random import choice

from annsa.template_sampling import *

In [2]:
import tensorflow.contrib.eager as tfe

In [3]:
tf.enable_eager_execution()

#### Import model, training function 

In [4]:
from annsa.model_classes import (dnn_model_features,
                                 DNN,
                                 save_model,
                                 train_earlystop)

## Load testing dataset 

In [5]:
dataset = np.load('../dataset_generation/testing_dataset_200keV_1000.npy')

In [6]:
all_spectra = np.add(dataset.item()['sources'], dataset.item()['backgrounds'])
all_keys = dataset.item()['keys']

mlb=LabelBinarizer()

all_keys_binarized = mlb.fit_transform(all_keys)

# Train network

### Define hyperparameters

In [7]:
number_hyperparameters_to_search = 256
earlystop_errors_test = []
model_id='DNN-kfoldseasy'

In [8]:
def make_model():
    '''
    Makes a random model given some parameters.

    '''
    number_layers = choice([1, 2, 3])
    dense_nodes = 2**np.random.randint(5, 10, number_layers)
    dense_nodes = np.sort(dense_nodes)
    dense_nodes = np.flipud(dense_nodes)
    model_features = dnn_model_features(
        learining_rate=10**np.random.uniform(-4,-1),
        l2_regularization_scale=10**np.random.uniform(-2,0),
        dropout_probability=np.random.uniform(0,1),
        batch_size=2**np.random.randint(4,10),
        output_size=all_keys_binarized.shape[1],
        dense_nodes=dense_nodes,
        activation_function=choice([tf.nn.tanh,tf.nn.relu,tf.nn.sigmoid]),
        scaler=choice([make_pipeline(FunctionTransformer(np.log1p, validate=True)),
                       make_pipeline(FunctionTransformer(np.sqrt, validate=True))]))

    model = DNN(model_features)

    return model, model_features 

### Search hyperparameters

In [ ]:
def save_features(model_features,
                  model_id,
                  hyperparameter_index):
    with open('./hyperparameter-search-results/' + model_id + '_' +
              str(hyperparameter_index) + '_dae_features', 'wb+') as f:
        pickle.dump(model_features,f)
    return None

skf = StratifiedKFold(n_splits=5, random_state=5)
testing_errors = []
all_kf_errors = []

for network_id in range(number_hyperparameters_to_search):
    print(network_id)
    model, model_features = make_model()
    save_features(model_features,
                  model_id,
                  network_id)
    
    k_folds_errors = []
    for train_index, test_index in skf.split(all_spectra, all_keys):
        # reset model on each iteration
        model = DNN(model_features)
        optimizer = tf.train.AdamOptimizer(model_features.learining_rate)

        costfunction_errors_tmp, earlystop_errors_tmp = train_earlystop(
                training_data=all_spectra[train_index],
                training_keys=all_keys_binarized[train_index],
                testing_data=all_spectra[test_index],
                testing_keys=all_keys_binarized[test_index],
                model=model,
                optimizer=optimizer,
                num_epochs=200,
                obj_cost=model.cross_entropy,
                earlystop_cost_fn=model.f1_error,
                earlystop_patience=10,
                not_learning_patience=10,
                not_learning_threshold=0.9,
                verbose=True,
                fit_batch_verbose=10,
                data_augmentation=model.default_data_augmentation)
        k_folds_errors.append(earlystop_errors_tmp)
        all_kf_errors.append(earlystop_errors_tmp)

    testing_errors.append(np.average(k_folds_errors))
    np.save('./final-models/final_test_errors_'+model_id, testing_errors)
    np.save('./final-models/final_kf_errors_'+model_id, all_kf_errors)
    # model.save_weights('./final-models/'+model_id+'_checkpoint_'+str(network_id))
    network_id += 1 

0
Epoch 10: CostFunc loss: 0.00 2.72, EarlyStop loss: 0.00 0.29
Epoch 20: CostFunc loss: 0.00 2.05, EarlyStop loss: 0.00 0.30
Epoch 30: CostFunc loss: 0.00 1.77, EarlyStop loss: 0.00 0.20
Epoch 40: CostFunc loss: 0.00 1.54, EarlyStop loss: 0.00 0.20
Epoch 50: CostFunc loss: 0.00 1.47, EarlyStop loss: 0.00 0.21
Test error at early stop: Objectives fctn: 1.46 Early stopfctn: 1.46
Epoch 10: CostFunc loss: 0.00 2.65, EarlyStop loss: 0.00 0.30
Epoch 20: CostFunc loss: 0.00 2.00, EarlyStop loss: 0.00 0.31
Epoch 30: CostFunc loss: 0.00 1.65, EarlyStop loss: 0.00 0.21
Test error at early stop: Objectives fctn: 1.83 Early stopfctn: 1.83
Epoch 10: CostFunc loss: 0.00 2.62, EarlyStop loss: 0.00 0.30
Epoch 20: CostFunc loss: 0.00 1.95, EarlyStop loss: 0.00 0.22
Epoch 30: CostFunc loss: 0.00 1.69, EarlyStop loss: 0.00 0.20
Test error at early stop: Objectives fctn: 1.79 Early stopfctn: 1.79
Epoch 10: CostFunc loss: 0.00 2.66, EarlyStop loss: 0.00 0.29
Epoch 20: CostFunc loss: 0.00 2.01, EarlyStop l

Epoch 50: CostFunc loss: 0.00 1.96, EarlyStop loss: 0.00 0.45
Epoch 60: CostFunc loss: 0.00 1.94, EarlyStop loss: 0.00 0.50
Epoch 70: CostFunc loss: 0.00 1.96, EarlyStop loss: 0.00 0.52
Test error at early stop: Objectives fctn: 1.87 Early stopfctn: 1.87
Epoch 10: CostFunc loss: 0.00 2.61, EarlyStop loss: 0.00 0.59
Epoch 20: CostFunc loss: 0.00 2.29, EarlyStop loss: 0.00 0.51
Epoch 30: CostFunc loss: 0.00 2.12, EarlyStop loss: 0.00 0.48
Epoch 40: CostFunc loss: 0.00 2.10, EarlyStop loss: 0.00 0.50
Epoch 50: CostFunc loss: 0.00 1.94, EarlyStop loss: 0.00 0.46
Epoch 60: CostFunc loss: 0.00 2.16, EarlyStop loss: 0.00 0.64
Test error at early stop: Objectives fctn: 1.88 Early stopfctn: 1.88
Epoch 10: CostFunc loss: 0.00 2.72, EarlyStop loss: 0.00 0.69
Epoch 20: CostFunc loss: 0.00 2.30, EarlyStop loss: 0.00 0.53
Test error at early stop: Objectives fctn: 2.31 Early stopfctn: 2.31
Epoch 10: CostFunc loss: 0.00 2.86, EarlyStop loss: 0.00 0.78
Epoch 20: CostFunc loss: 0.00 2.31, EarlyStop los

Epoch 30: CostFunc loss: 0.00 1.41, EarlyStop loss: 0.00 0.31
Epoch 40: CostFunc loss: 0.00 1.34, EarlyStop loss: 0.00 0.27
Test error at early stop: Objectives fctn: 1.12 Early stopfctn: 1.12
Epoch 10: CostFunc loss: 0.00 1.64, EarlyStop loss: 0.00 0.33
Epoch 20: CostFunc loss: 0.00 1.18, EarlyStop loss: 0.00 0.20
Epoch 30: CostFunc loss: 0.00 1.16, EarlyStop loss: 0.00 0.20
Epoch 40: CostFunc loss: 0.00 1.06, EarlyStop loss: 0.00 0.22
Test error at early stop: Objectives fctn: 1.08 Early stopfctn: 1.08
Epoch 10: CostFunc loss: 0.00 1.58, EarlyStop loss: 0.00 0.26
Epoch 20: CostFunc loss: 0.00 1.20, EarlyStop loss: 0.00 0.23
Epoch 30: CostFunc loss: 0.00 1.05, EarlyStop loss: 0.00 0.22
Epoch 40: CostFunc loss: 0.00 0.96, EarlyStop loss: 0.00 0.19
Test error at early stop: Objectives fctn: 0.99 Early stopfctn: 0.99
15
Epoch 10: CostFunc loss: 0.00 1587.41, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 277.74 Early stopfctn: 277.74
Epoch 10: CostFunc loss: 0.00 14

Epoch 70: CostFunc loss: 0.00 1.22, EarlyStop loss: 0.00 0.14
Epoch 80: CostFunc loss: 0.00 1.11, EarlyStop loss: 0.00 0.13
Epoch 90: CostFunc loss: 0.00 1.02, EarlyStop loss: 0.00 0.12
Epoch 100: CostFunc loss: 0.00 0.96, EarlyStop loss: 0.00 0.12
Epoch 110: CostFunc loss: 0.00 0.91, EarlyStop loss: 0.00 0.11
Epoch 120: CostFunc loss: 0.00 0.85, EarlyStop loss: 0.00 0.11
Epoch 130: CostFunc loss: 0.00 0.81, EarlyStop loss: 0.00 0.11
Epoch 140: CostFunc loss: 0.00 0.77, EarlyStop loss: 0.00 0.10
Test error at early stop: Objectives fctn: 0.77 Early stopfctn: 0.77
Epoch 10: CostFunc loss: 0.00 4.03, EarlyStop loss: 0.00 0.46
Epoch 20: CostFunc loss: 0.00 2.88, EarlyStop loss: 0.00 0.28
Epoch 30: CostFunc loss: 0.00 2.23, EarlyStop loss: 0.00 0.19
Epoch 40: CostFunc loss: 0.00 1.82, EarlyStop loss: 0.00 0.17
Epoch 50: CostFunc loss: 0.00 1.55, EarlyStop loss: 0.00 0.14
Epoch 60: CostFunc loss: 0.00 1.34, EarlyStop loss: 0.00 0.13
Epoch 70: CostFunc loss: 0.00 1.20, EarlyStop loss: 0.00 0